# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key; use your own API keys in config.py
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# file path
file = "../output_data/cities.csv"

# read csv file to dataframe
cities_data = pd.DataFrame(pd.read_csv(file, index_col="City ID"))

# display dataframe
cities_data


,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City ID,,,,,,,,,
0,Cabo San Lucas,MX,2021-10-23 21:11:22,22.8909,-109.9124,92.52,61,20,17.27
1,Boa Vista,BR,2021-10-23 21:13:48,2.8197,-60.6733,91.38,49,40,6.91
2,Ushuaia,AR,2021-10-23 21:09:04,-54.8000,-68.3000,33.46,93,75,29.93
3,Vila Velha,BR,2021-10-23 21:11:10,-20.3297,-40.2925,75.15,78,75,12.66
4,Rikitea,PF,2021-10-23 21:10:37,-23.1203,-134.9692,74.79,82,99,21.36
...,...,...,...,...,...,...,...,...,...
562,The Valley,AI,2021-10-23 21:08:59,18.2170,-63.0578,83.10,70,2,16.89
563,Richards Bay,ZA,2021-10-23 21:11:18,-28.7830,32.0377,70.59,64,83,20.87
564,Mathbaria,BD,2021-10-23 21:15:54,22.2876,89.9593,72.23,77,38,4.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# config gmaps api key
gmaps.configure(api_key=g_key)

# use Lat and Lng as locations; Humidity as weight
locations = cities_data[["Lat", "Lng"]]
weights = cities_data["Humidity (%)"]

# formatting
figure_layout = {
    'width': "100%",
    'height': '400px',
#    'border': '1px solid black',
    'padding': '1px',
#    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
fig.add_layer(heatmap_layer)

# set maximum intensity and point_radius settings
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

# display heatmap
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the DataFrame to find your ideal weather condition. For example
# 1. A max temperature lower than 80 degrees but higher than 70.
# 2. Wind speed less than 10 mph.
# 3. Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_cities = cities_data.loc[(cities_data["Max Temp (F)"] > 70) & 
                               (cities_data["Max Temp (F)"] < 80) & 
                               (cities_data["Wind Speed (mph)"] < 10) & 
                               (cities_data["Cloudiness (%)"] == 0) &
                               (cities_data["Humidity (%)"] < 75), :].reset_index(drop=True).dropna()
ideal_cities

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Wilmington,US,2021-10-23 21:04:25,34.2257,-77.9447,75.97,71,0,1.99
1,Havelock,US,2021-10-23 21:14:24,34.8791,-76.9013,76.95,61,0,1.01
2,Ovalle,CL,2021-10-23 21:15:08,-30.5983,-71.2003,74.53,31,0,8.66
3,Mau,IN,2021-10-23 21:15:29,25.9500,83.5500,72.18,66,0,3.13
4,Salalah,OM,2021-10-23 21:11:29,17.0151,54.0924,75.29,60,0,5.75
5,Paine,CL,2021-10-23 21:15:41,-33.8167,-70.7500,78.30,20,0,5.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# store data

hotel_df = ideal_cities[["City", "Country", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to the DataFrame with null values
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Wilmington,US,34.2257,-77.9447,NaN
1,Havelock,US,34.8791,-76.9013,NaN
2,Ovalle,CL,-30.5983,-71.2003,NaN
3,Mau,IN,25.9500,83.5500,NaN
4,Salalah,OM,17.0151,54.0924,NaN
5,Paine,CL,-33.8167,-70.7500,NaN


In [14]:
# parameters for querying Google Places API
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [15]:
# loop through each city in the dataframe and make an API call

for index, row in hotel_df.iterrows():
    
    # get coordinates and store in dictionary
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # call the API and get hotel details for each city
    hotel_data = requests.get(base_url, params).json()
    
    # exception handling to save the first hotel name for each city
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("-----------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-----------------------------------------------------------------")

City 'Wilmington' found! The nearest hotel is Hotel Ballast Wilmington, Tapestry Collection by Hilton
-------------------------------------------------------------------------
City 'Havelock' found! The nearest hotel is Hampton Inn Havelock
-------------------------------------------------------------------------
City 'Ovalle' found! The nearest hotel is Hotel Keo
-------------------------------------------------------------------------
City 'Mau' found! The nearest hotel is Gitanjali Hotel
-------------------------------------------------------------------------
City 'Salalah' found! The nearest hotel is Salalah Gardens Hotel
-------------------------------------------------------------------------
City 'Paine' found! The nearest hotel is Hotel El Almendro
-------------------------------------------------------------------------


In [16]:
# drop null from dataframe
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# display dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Wilmington,US,34.2257,-77.9447,"Hotel Ballast Wilmington, Tapestry Collection ..."
1,Havelock,US,34.8791,-76.9013,Hampton Inn Havelock
2,Ovalle,CL,-30.5983,-71.2003,Hotel Keo
3,Mau,IN,25.9500,83.5500,Gitanjali Hotel
4,Salalah,OM,17.0151,54.0924,Salalah Gardens Hotel
5,Paine,CL,-33.8167,-70.7500,Hotel El Almendro


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))